# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import altair as alt

In [ ]:
supplier = pd.read_excel('../input/brothers-in-arms/Arms-Companies-2015-2019.xlsx')
exporter = pd.read_csv('../input/brothers-in-arms/Arms-Export-by-supplier-1950-2020.csv')
importer = pd.read_csv('../input/brothers-in-arms/Arms-Import-by-country-1950-2020.csv')
weapon = pd.read_csv('../input/brothers-in-arms/Arms-Export-by-weapon-1950-2020.csv')

#### Converting data from wide to long format, excluding country which had trade lesser than 100 millions of dollar for country wise analysis (doesn't impact the total trade volumes)

In [ ]:
exporter = pd.melt(exporter,id_vars=['Country'],value_vars=exporter.columns[1:])
exporter = exporter.fillna(0)
exporter['value'] = exporter['value'].astype(int)
exporter = exporter[exporter['value']>100]

importer = pd.melt(importer,id_vars=['Country'],value_vars=importer.columns[1:])
importer = importer.fillna(0)
importer['value'] = importer['value'].astype(int)
importer = importer[importer['value']>100]

weapon = pd.melt(weapon,id_vars=['Weapon'],value_vars=weapon.columns[1:])
weapon = weapon.fillna(0)
weapon['value'] = weapon['value'].astype(int)
weapon = weapon[weapon['value']>100]

supplier = pd.melt(supplier,id_vars=['Country', 'Company'],value_vars=supplier.columns[2:])

## Total Arms traded in world by value and year 1950-2020

In [ ]:
total_export = exporter[exporter['Country']=='Total']
alt.Chart(total_export[total_export['variable']!='Total'], title='Total Arms export since 1950-2019').mark_line(point=True).encode(
    alt.X('variable', title='Year',axis=alt.Axis(labelAngle=-45)),
    alt.Y('value', title='Export in millions of $')
).properties(height=300, width=900)

In [ ]:
exporter = exporter[exporter['Country']!='Total']
importer = importer[importer['Country']!='Total']
weapon = weapon[weapon['Weapon']!='Total']

biggest_exporter = exporter[exporter['variable']=='Total']
biggest_importer = importer[importer['variable']=='Total']
biggest_weapon = weapon[weapon['variable']=='Total']

## Total Arms export by country

In [ ]:
bar = alt.Chart(biggest_exporter, title='Biggest Arms Exporting Countries').mark_bar().encode(
    alt.X('Country', title='Country', axis=alt.Axis(labelAngle=-45)),
    alt.Y('value', title='Export in millions of $'),
    color=alt.condition(
        alt.datum.Country == 'United States',
        alt.value('orange'),     
        alt.value('steelblue')
    )
)
rule = alt.Chart(biggest_exporter).mark_rule(color='red').encode(
    y='mean(value)'
)
(bar + rule).properties(width=800)

#### Without doubt USA came to be number 1, Followed by Soviet union till 1990 when it supplied arms to all east european nations during cold war till Soviet union fell

## Total Arms Import by country

In [ ]:
bar = alt.Chart(biggest_importer, title='Biggest Arms Importing Countries').mark_bar().encode(
    alt.X('Country', axis=alt.Axis(labelAngle=-45)),
    alt.Y('value', title='Import in millions of $'),
    color=alt.condition(
        alt.datum.Country == 'India',  
        alt.value('orange'),     
        alt.value('steelblue')  
    )
)
rule = alt.Chart(biggest_importer).mark_rule(color='red').encode(
    y='mean(value)'
)
(bar + rule).properties(width=1400)

#### India has been biggest importer of arms followed by china

## Total Arms trade by value

In [ ]:
bar = alt.Chart(biggest_weapon, title='Trade of Arms by type').mark_bar().encode(
    alt.X('Weapon', axis=alt.Axis(labelAngle=-45)),
    alt.Y('value', title='Value in millions of $'),
    color=alt.condition(
        alt.datum.Weapon == 'Aircraft',  
        alt.value('orange'),    
        alt.value('steelblue')
    )
)
rule = alt.Chart(biggest_weapon).mark_rule(color='red').encode(
    y='mean(value)'
)
(bar + rule).properties(width=800)

#### It seems world arms trade is biased towards good ari capabilities

- Excluding rows with total as value with years for country wise analysis

In [ ]:
exporter = exporter[exporter['variable']!='Total']
importer = importer[importer['variable']!='Total']
weapon = weapon[weapon['variable']!='Total']

## Arms export over the years by country

In [ ]:
dropdown_category = alt.binding_select(options=list(exporter["Country"].unique()), name=" ")
selection_country = alt.selection_single(fields=["Country"], bind=dropdown_category, init={'Country': 'Russia'})

alt.Chart(exporter, title='Year wise arms export 1950-2020').mark_line(point=True).encode(
    alt.X('variable', title='Year', axis=alt.Axis(labelAngle=-45)),
    alt.Y('value',title='Value in milliosn of $', scale=alt.Scale(type='linear', domain=[100, 20000])),
    color='Country',
    opacity=alt.condition(
        selection_country,
        alt.value(1),
        alt.value(0.1)
)).add_selection(selection_country).properties(height=500, width=800)

***Dropdown avaialble up***

#### Look at the sharp fall of soviet union.

## Arms Import over the years by country

In [ ]:
dropdown_category = alt.binding_select(options=list(importer["Country"].unique()), name=" ")
selection_country = alt.selection_single(fields=["Country"], bind=dropdown_category,init={'Country': 'India'})

alt.Chart(importer, title='Year wise arms import 1950-2020').mark_line(point=True).encode(
    alt.X('variable', title='Year',  axis=alt.Axis(labelAngle=-45)),
    alt.Y('value',title='Value in millions of $',scale=alt.Scale(type='linear', domain=[100, 6000])),
    color='Country',
    opacity=alt.condition(
        selection_country,
        alt.value(1),
        alt.value(0.1)
)).add_selection(selection_country).properties(height=500, width=800)

***Dropdown avaialble***

####  India is build its arms capability since last decade it seems after the cooldown period of 1990-2005. Interesting thing to look at would be countries like afganistan, syria, etc which were war torn since 2000.

## Arms Trade over the years by Weapon

In [ ]:
dropdown_category = alt.binding_select(options=list(weapon["Weapon"].unique()), name=" ")
selection_weapon = alt.selection_single(fields=["Weapon"], bind=dropdown_category, init={'Weapon': 'Armoured vehicles'})

alt.Chart(weapon, title='Arms export over the year by weapon type').mark_line(point=True).encode(
    alt.X('variable', title='Year',  axis=alt.Axis(labelAngle=-45)),
    alt.Y('value',title= 'Value in millions of $', scale=alt.Scale(type='linear', domain=[100, 20000])),
    color='Weapon',
    opacity=alt.condition(
        selection_weapon,
        alt.value(1),
        alt.value(0.1)
)).add_selection(selection_weapon).properties(height=500, width=800)

***Dropdown avaialble***

#### It looks like armoured vehicles and artillary are losing their importance in arms trade 

## Arms producing companies by sales value

In [ ]:
dropdown_category = alt.binding_select(options=list(supplier["Company"].unique()), name=" ")
selection_company = alt.selection_single(fields=["Company"], bind=dropdown_category,init={'Company': 'Lockheed Martin Corp.'})

alt.Chart(supplier, title='Biggest Arms Manufacturer 2015-2019').mark_line(point=True).encode(
    alt.X('variable:O', title='Year',  axis=alt.Axis(labelAngle=-45)),
    alt.Y('value', title='Sales in millions of $'),
    color='Company',
    opacity=alt.condition(
        selection_company,
        alt.value(1),
        alt.value(0.1)
)).add_selection(selection_company).properties(height=600, width=800)

***Dropdown avaialble***

#### Didn't know arms trade is so big. Looking at top companies they all make aircrafts. 